In [2]:
import multiprocess as mp
num_nucleos = mp.cpu_count()
print(num_nucleos)

8


In [12]:
import  pandas as pd
df_csv_original = pd.read_csv('credit_data.csv')

df_csv_original.dropna(inplace=True)
df_csv_original = df_csv_original[df_csv_original['Gender'] != 'Other']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df_csv_original['Gender'] = le.fit_transform(df_csv_original['Gender'])
df_csv_original['Existing Customer'] = le.fit_transform(df_csv_original['Existing Customer'])
df_csv_original['State'] = le.fit_transform(df_csv_original['State'])
df_csv_original['City'] = le.fit_transform(df_csv_original['City'])
df_csv_original['Employment Profile'] = le.fit_transform(df_csv_original['Employment Profile'])
df_csv_original['Occupation'] = le.fit_transform(df_csv_original['Occupation'])
df_csv_original['Profile Score'] = df_csv_original['Profile Score'].apply(lambda x: 1 if x >= 80 else 0)
display(df_csv_original.head())

df_csv_original.to_csv('credit_dataCLEAN.csv', index=False)

,Age,Gender,Income,Credit Score,Credit History Length,Number of Existing Loans,Loan Amount,Loan Tenure,Existing Customer,State,City,LTV Ratio,Employment Profile,Profile Score,Occupation
0,31,1,36000,604,487,5,109373,221,0,2,15,90.943430,1,0,4
1,25,1,50000,447,386,2,150000,89,0,2,1,91.135253,1,0,10
3,69,0,46000,668,349,6,150000,148,1,2,1,87.393365,2,1,3
4,52,1,132000,601,553,5,150000,157,0,2,15,66.158757,1,1,12
5,64,0,127000,850,158,10,108702,111,1,6,5,82.331250,2,1,3


In [ ]:
import itertools
import pandas as pd
import multiprocess
import time

# Cargar el dataset
df = pd.read_csv('credit_dataCLEAN.csv')
df = df.head(100000)

# Definir la cuadrícula de hiperparámetros para Random Forest
param_grid_rf = {
    'n_estimators': [x for x in range(10, 101, 5)],  
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_features': ['sqrt', 'log2'],  
    'warm_start': [True, False]
}

# Generar todas las combinaciones de hiperparámetros
keys_rf, values_rf = zip(*param_grid_rf.items())
combinations_rf = [dict(zip(keys_rf, v)) for v in itertools.product(*values_rf)]

# Función para nivelar cargas
def nivelacion_cargas(D, n_p):
    t = len(D) // n_p  
    r = len(D) % n_p   

    out = []
    start = 0
    for i in range(n_p):
        end = start + t + (1 if i < r else 0)
        out.append(D[start:end])
        start = end
    
    return out

# Función para evaluar el conjunto de hiperparámetros
def evaluate_set(df, namecolum, hyperparameter_set, lock):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score

    X = df.drop(columns=[namecolum])  
    y = df[namecolum]  
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.20)
    
    for s in hyperparameter_set:
        clf = RandomForestClassifier()
        clf.set_params(
            n_estimators=s['n_estimators'], 
            criterion=s['criterion'], 
            max_features=s['max_features'], 
            warm_start=s['warm_start']
        )
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        lock.acquire()
        try:
            print(f"{s} \t Accuracy:{accuracy_score(y_test, y_pred)}")
        finally:
            lock.release()

# Ejecución principal
if __name__ == '__main__':
    N_THREADS = 2
    
    splits = nivelacion_cargas(combinations_rf, N_THREADS)

    for i, split in enumerate(splits):
        print(f"Proceso {i+1} tiene {len(split)} combinaciones.")

    lock = multiprocess.Lock()
    
    threads = []
    
    for i in range(N_THREADS):
        threads.append(multiprocess.Process(target=evaluate_set, args=(df, 'Profile Score', splits[i], lock)))

    start_time = time.perf_counter()
    
    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()
    
    finish_time = time.perf_counter()
    print(f"Program finished in {finish_time - start_time} seconds")


In [3]:
import pandas as pd
dfResults = pd.read_csv('timeResultsRF.csv')

dfResults['media'] = dfResults.iloc[:, 1:].mean(axis=1)
dfResults['desEstandar'] = dfResults.iloc[:, 1:].std(axis=1)

dfResults.to_csv('timeResultsRF.csv', index=False)

In [2]:
import pandas as pd
dfResults = pd.read_csv('timeResultsRF.csv')
df = pd.DataFrame(dfResults)

# Calcular speedup (comparando cada tiempo promedio con el tiempo promedio para 1 hilo)
df['speedup'] = df['media'].iloc[0] / df['media']

# Calcular eficiencia (speedup / número de hilos)
df['eficiencia'] = df['speedup'] / df['threads']

df.to_csv('timeResultsRF.csv', index=False)


In [3]:
import pandas as pd
dfResults = pd.read_csv('resultadosKNN.csv')
df = pd.DataFrame(dfResults)

# Calcular speedup (comparando cada tiempo promedio con el tiempo promedio para 1 hilo)
df['speedup'] = df['Media'].iloc[0] / df['Media']

# Calcular eficiencia (speedup / número de hilos)
df['eficiencia'] = df['speedup'] / df['No. Hilos']

df.to_csv('resultadosKNN.csv', index=False)
